# 영업 성공 여부 분류 경진대회 

# 예시코드

In [408]:
!pip install xgboost
!pip install impyute
!pip install lightgbm
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [410]:

import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from impyute.imputation.cs import mice
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.impute import SimpleImputer
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [411]:

df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)



In [412]:
# 결측값 개수 확인 : df_train['com_reg_ver_win_rate'].isnull().sum()

In [413]:
#값 분포 확인 : df_train['lead_owner'].unique()
#df_train.info(0)

# 무의미 피처 삭제

In [414]:
def remove_feature(df, col):
    df.drop(columns=col,inplace=True)
    return df

#1. customer_country1. 삭제
remove_feature(df_train,["customer_country.1"])
remove_feature(df_test,["customer_country.1"])

#2. product_subcategory 삭제
remove_feature(df_train,["product_subcategory"])
remove_feature(df_test,["product_subcategory"])


#3. product_modelname 삭제
remove_feature(df_train,["product_modelname"])
remove_feature(df_test,["product_modelname"])

#4. business_subarea 삭제
remove_feature(df_train,["business_subarea"])
remove_feature(df_test,["business_subarea"])

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,NaN,...,none,LGESP,NaN,1,0,0.001183,0.049840,retail,278,True
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,NaN,NaN,...,none,LGEUS,NaN,0,0,0.000013,NaN,transportation,437,True
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,NaN,...,manager,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,874,False
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,NaN,...,none,LGEUS,more than a year,0,0,0.001183,0.049840,retail,194,False
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,1.0,...,others,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,167,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,13855,0.50,/São Paulo/Brazil,AS,NaN,40292,NaN,Enterprise,10.0,NaN,...,associate/analyst,LGESP,NaN,0,0,NaN,NaN,NaN,97,False
5267,7979,0.25,General / / United States,IT,NaN,47466,NaN,Enterprise,0.0,NaN,...,none,LGEUS,NaN,0,0,NaN,NaN,NaN,438,True
5268,12887,0.75,/ OURO BRANCO / Brazil,AS,NaN,46227,Specifier/ Influencer,Enterprise,NaN,NaN,...,manager,LGESP,less than 3 months,0,0,NaN,NaN,NaN,97,False
5269,17530,0.00,/ / Germany,IT,NaN,45667,End Customer,SMB,NaN,NaN,...,none,LGEDG,NaN,0,0,NaN,NaN,NaN,429,True


In [415]:
#df_train.info()

# 이상치 처리

In [416]:
def replace_outliers(df, col_name):
    Q1 = df[col_name].quantile(0.25)
    Q3 = df[col_name].quantile(0.75)
    IQR = Q3 - Q1

    # 상하한값 설정
    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR

    # 이상치를 상하한값으로 대체
    df[col_name] = df[col_name].apply(lambda x: min(max(x, lower_limit), upper_limit))
    
    return df


In [417]:
#1. lead_desc_length에 대해서 이상치 상하한값 대체
replace_outliers(df_train,'lead_desc_length')
replace_outliers(df_test,'lead_desc_length')

#2. ver_win_rate_x에 대해서 이상치 상하한값 대체
replace_outliers(df_train,'ver_win_rate_x')
replace_outliers(df_test,'ver_win_rate_x')

#3. ver_win_ratio_per_bu에 대해서 이상치 상하한값 대체
replace_outliers(df_train,'ver_win_ratio_per_bu')
replace_outliers(df_test,'ver_win_ratio_per_bu')


,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,NaN,...,none,LGESP,NaN,1,0,0.001183,0.049840,retail,278,True
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,NaN,NaN,...,none,LGEUS,NaN,0,0,0.000013,NaN,transportation,437,True
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,NaN,...,manager,LGEGF,less than 3 months,0,0,0.000060,0.088471,hospital & health care,874,False
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,NaN,...,none,LGEUS,more than a year,0,0,0.001183,0.049840,retail,194,False
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,1.0,...,others,LGESP,less than 3 months,1,1,0.002866,0.064566,corporate / office,167,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,13855,0.50,/São Paulo/Brazil,AS,NaN,40292,NaN,Enterprise,10.0,NaN,...,associate/analyst,LGESP,NaN,0,0,NaN,NaN,NaN,97,False
5267,7979,0.25,General / / United States,IT,NaN,47466,NaN,Enterprise,0.0,NaN,...,none,LGEUS,NaN,0,0,NaN,NaN,NaN,438,True
5268,12887,0.75,/ OURO BRANCO / Brazil,AS,NaN,46227,Specifier/ Influencer,Enterprise,NaN,NaN,...,manager,LGESP,less than 3 months,0,0,NaN,NaN,NaN,97,False
5269,17530,0.00,/ / Germany,IT,NaN,45667,End Customer,SMB,NaN,NaN,...,none,LGEDG,NaN,0,0,NaN,NaN,NaN,429,True


In [418]:

"""
# 박스 플롯으로 이상치 확인
plt.figure(figsize=(10, 6))
sns.boxplot(df_train['ver_win_rate_x'])
plt.show()
print(df_train['lead_owner'].describe())
print(df_train['lead_desc_length'].describe())
"""

"\n# 박스 플롯으로 이상치 확인\nplt.figure(figsize=(10, 6))\nsns.boxplot(df_train['ver_win_rate_x'])\nplt.show()\nprint(df_train['lead_owner'].describe())\nprint(df_train['lead_desc_length'].describe())\n"

In [419]:
# 기간 범주화 및 결측치 처리
def simplify_expected_timeline(row):
    if pd.isna(row):
        return 'Unknown'
    elif 'less than 3 months' in row:
        return 'less than 3 months'
    elif '3 months ~ 6 months' in row:
        return '3 months ~ 6 months'
    elif '6 months ~ 9 months' in row:
        return '6 months ~ 9 months'
    elif '9 months ~ 1 year' in row:
        return '9 months ~ 1 year'
    elif 'more than a year' in row:
        return 'more than a year'
    else:
        return 'Unknown'



# 결측치 처리

In [420]:
def impute_missing_values(df):
    # 1-1. com_reg_ver_win_rate의 결측치를 lead_owner의 분포를 이용해 중앙값으로 대체
    com_reg_missing = df.groupby('lead_owner')['com_reg_ver_win_rate'].median()
    for lead_owner, median_value in com_reg_missing.items():
        df.loc[(df['lead_owner'] == lead_owner) & (df['com_reg_ver_win_rate'].isnull()), 'com_reg_ver_win_rate'] = median_value

    # 1-2. com_reg_ver_win_rate에 대해 처리되지 않은 결측치들은 전체 중앙값으로 대체
    overall_median_com_reg_missing = df['com_reg_ver_win_rate'].median()
    df['com_reg_ver_win_rate'].fillna(overall_median_com_reg_missing, inplace=True)
    
    # 2. historical_existing_cnt에 대한 결측치 0으로
    df['historical_existing_cnt'].fillna(0, inplace=True)

    # 3. 가중치를 나타내는 세 피처에 대해 결측값을 0으로
    df['id_strategic_ver'].fillna(0, inplace=True)
    df['it_strategic_ver'].fillna(0, inplace=True)
    df['idit_strategic_ver'].fillna(0, inplace=True)

    # 4. ver_pro와 ver_win_rate_x의 상관관계를 통해 결측치 중앙값으로 대체
    median_ver_win_rate_x_for_ver_pro_0 = df.loc[df['ver_pro'] == 0, 'ver_win_rate_x'].median()
    median_ver_win_rate_x_for_ver_pro_1 = df.loc[df['ver_pro'] == 1, 'ver_win_rate_x'].median()
    df.loc[(df['ver_win_rate_x'].isnull()) & (df['ver_pro'] == 0), 'ver_win_rate_x'] = median_ver_win_rate_x_for_ver_pro_0
    df.loc[(df['ver_win_rate_x'].isnull()) & (df['ver_pro'] == 1), 'ver_win_rate_x'] = median_ver_win_rate_x_for_ver_pro_1

    # 5-1. ver_win_ratio_per_bu에 대해 결측값은 business unit별 중앙값으로
    bu_median_ver_win_ratio_per_bu = df.groupby('business_unit')['ver_win_ratio_per_bu'].median()
    for business_unit, median_value in bu_median_ver_win_ratio_per_bu.items():
        df.loc[(df['business_unit'] == business_unit) & (df['ver_win_ratio_per_bu'].isnull()), 'ver_win_ratio_per_bu'] = median_value

    # 5-2. ver_win_ratio_per_bu에 대해 처리되지 않은 결측값들은 전체 중앙값으로 대체
    overall_median_ver_win_ratio_per_bu = df['ver_win_ratio_per_bu'].median()
    df['ver_win_ratio_per_bu'].fillna(overall_median_ver_win_ratio_per_bu, inplace=True)
    
    #6 customer_country OT로 결측값 대체
    df['customer_country'].fillna('OT', inplace=True)
    
    #7-1 customer-type에서 유사 카테고리 통합
    df['customer_type'] = df['customer_type'].replace({
        'End Customer': 'End-Customer',
        'Specifier/ Influencer': 'Specifier/Influencer',
        'Specifier / Influencer': 'Specifier/Influencer',
        'Home Owner': 'Homeowner',
        'End-user': 'End-User',
        'Software/Solution Provider': 'Software/Solution Provider',
        'Software / Solution Provider': 'Software/Solution Provider',
        'Others': 'Other',
        'Dealer/Distributor': 'Distributor',
    
        })
    
    #7-2 customer_type Other로 결측값 대체
    df['customer_type'].fillna('Other', inplace=True)
    
    #8-1 customer_job other로 결측값 대체
    df['customer_job'].fillna('other', inplace=True)
    
    #8-2 customer_job에서 매우 낮은 빈도로 출현하는 직업을 other로 대체
    customer_job_distribution = df['customer_job'].value_counts(dropna=False)
    top_n_categories = customer_job_distribution.index[:10]  # 상위 10개 카테고리
    df['customer_job'] = df['customer_job'].apply(lambda x: x if x in top_n_categories else 'other')
    
    # 9-1 inquiry_type에대해서 대소문자 통일, 유사 카테고리 통합
    df['inquiry_type'] = df['inquiry_type'].str.lower()
    df['inquiry_type'] = df['inquiry_type'].replace({
        'quotation or purchase consultation': 'quotation or purchase consultation',  
        })
    #9-2 inquiry_type에 대해서 결측값 other로 대체
    df['inquiry_type'].fillna('other', inplace=True)
    
    #10 product_cateogry에 대해서 결측값 other로 대체
    df['product_category'].fillna('other', inplace=True)
    
    #11 expected_timeline 기간 통일하고, 결측값 unknown으로 대체
    df['expected_timeline'] = df['expected_timeline'].apply(simplify_expected_timeline)

    #12 business_area 결측값 other로 대체
    df['business_area'].fillna('other', inplace=True)
    return df


In [421]:
df_train = impute_missing_values(df_train)
df_test = impute_missing_values(df_test)


In [422]:
#df_train['expected_timeline'].isnull().sum()
#df_train.info()
#value_counts = df_train['product_category'].value_counts(dropna=False)
#value_counts
#value_counts2 = df_train['business_area'].value_counts(dropna=False)
#value_counts2

In [423]:
#df_train.info()

In [424]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series


In [425]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "customer_position",
    "response_corporate",
    "expected_timeline",
    
]
df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])


In [426]:

for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]



In [427]:

x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

In [428]:
smote = SMOTE(random_state=1234)
X_resampled, y_resampled = smote.fit_resample(x_train, y_train)




# 그리드서칭 (랜덤서칭 추가)

In [616]:
# LGBMClassifier를 위한 RandomizedSearchCV 
lgbm_param_dist = {
    'n_estimators': [10, 20, 30],
    'learning_rate': [0.01, 0.03, 0.05], 
    'max_depth': [7, 9],
    'num_leaves': [8, 16, 31], 
    'lambda_l2': [0, 0.1, 1, 8], 
}

# XGBClassifier를 위한 RandomizedSearchCV 
xgb_param_dist = {
    'n_estimators': [10, 20, 30], 
    'learning_rate': [0.01, 0.03, 0.05],  
    'max_depth': [6, 8],  
    'subsample': [0.8, 1.0], 
    'reg_lambda': [0, 0.1, 1, 8],  

}

lgbm_random_search = RandomizedSearchCV(LGBMClassifier(random_state=1234), param_distributions=lgbm_param_dist, n_iter=2, cv=2, scoring='f1', n_jobs=-1, random_state=1234)
xgb_random_search = RandomizedSearchCV(XGBClassifier(random_state=1234), param_distributions=xgb_param_dist, n_iter=2, cv=2, scoring='f1', n_jobs=-1, random_state=1234)


lgbm_random_search.fit(X_resampled, y_resampled)
xgb_random_search.fit(X_resampled, y_resampled)


best_lgbm_model = lgbm_random_search.best_estimator_
best_xgb_model = xgb_random_search.best_estimator_


[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 43536, number of negative: 43536
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004124 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3423
[LightGBM] [Info] Number of data points in the train set: 87072, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


In [617]:

ensemble_model = VotingClassifier(estimators=[
    ('lgbm', best_lgbm_model),
    ('xgb', best_xgb_model)
], voting='soft')

In [618]:

ensemble_model.fit(X_resampled, y_resampled)

[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 43536, number of negative: 43536
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004535 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3423
[LightGBM] [Info] Number of data points in the train set: 87072, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


VotingClassifier(estimators=[('lgbm',
                              LGBMClassifier(lambda_l2=0, learning_rate=0.05,
                                             max_depth=9, n_estimators=10,
                                             random_state=1234)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            featur...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.05, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=6,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=10, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=1234, ...))],
                 voting='soft')

In [619]:

def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))



In [620]:
pred = ensemble_model.predict(x_val)

get_clf_eval(y_val, pred)


[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
오차행렬:
 [[ 750  197]
 [ 960 9953]]

정확도: 0.9024
정밀도: 0.4386
재현율: 0.7920
F1: 0.5645


In [621]:
x_test = df_test.drop(["is_converted", "id"], axis=1)



test_pred = ensemble_model.predict(x_test)

sum(test_pred) # True로 예측된 개수





[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


2162

In [518]:

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)
